In [ ]:
import numpy as np
import pandas as pd
import numba

# Unsteady flow : Collocated

In [ ]:
@numba.jit(nopython=True, parallel=False)
def UnSteadyflowStaggered(A, Q, Adown, Qup, zb, B, dx, dt, g, manning):
    imax = len(A)
    Anew, Qnew = np.zeros(imax), np.zeros(imax)
    Qhf, Qhfnew = np.zeros(imax), np.zeros(imax) # 下流端は使わないためimax個とする。
    ie = np.zeros(imax)
    Vhf = np.zeros(imax+1)
    
# Q value cell-center to harf cell
# すべてをセルセンターで扱うため、ハーフセルへの変換では逆変換との兼ね合いで次のように定義する。
    Qhf[-1] = Q[-1]
    for i in range(imax-2, -1, -1) : 
        Qhf[i] = 2.0*Q[i] - Qhf[i+1]
    
    for i in range(imax) : 
        ie[i] = manning**2 * Q[i]**2 / B[i]**2 / ( A[i]/B[i] )**(10.0/3.0)
    
# continuous equation
    for i in range(0, imax-1) : 
        Anew[i] = A[i] - dt * ( Qhf[i+1] - Qhf[i] ) / dx
    Anew[-1] = Adown
    
    Hnew = Anew/B + zb
    
    # 逆流の場合は風上が必要
    for i in range(1, imax) : 
        Vhf[i] = Qhf[i]/A[i-1]
    Vhf[0] = Qhf[0]/A[0]
    Vhf[-1] = Q[-1]/A[-1]
    
# moumentum equation
    for i in range(1, imax): 
        dHdx = ( Hnew[i] - Hnew[i-1] ) / dx
        Ahfnew = 0.5*(Anew[i] + Anew[i-1])
        Ahf = 0.5*(A[i]+A[i-1])
        iehf = 0.5*(ie[i]+ie[i-1])
        Vp = 0.5*(Vhf[i+1] + Vhf[i])
        Vm = 0.5*(Vhf[i-1] + Vhf[i])
        
        Qhfnew[i] = Qhf[i] - dt * ( Vp*Qhf[i] - Vm*Qhf[i-1] ) / dx \
                         - dt * g * Ahfnew * dHdx \
                         - dt * g * Ahf * iehf \
            
    Qhfnew[0] = Qup
    
    for i in range(imax-1): 
        Qnew[i] = 0.5*( Qhfnew[i+1] + Qhfnew[i] )
    Qnew[-1] = Qhfnew[-1]
    
    return Anew, Qnew, np.abs(Anew - A).max()

In [ ]:
# df = pd.read_csv('zb2.csv',index_col=0)
df = pd.read_csv('https://raw.githubusercontent.com/computational-sediment-hyd/1DSWEwithSmallerDx/main/zb2.csv',index_col=0)

In [ ]:
q = 10.0
n = 0.03
ib = 1/400
g = 9.8

h0 = (q**2*n**2/ib)**0.3 #等流水深
hc = (q**2/g)**(1/3) # 限界水深

## dx=0.5

In [ ]:
%%time

dx = 0.5
Ls1 = np.arange(0,10000.1,dx)
Zip1 = np.interp(Ls1, df.L, df.Z)

imax = len(Ls1)
dt = 0.025
totalTime = 3.0*3600.0
manning = n

# Initial & Boundary condition
zb = Zip1.copy()
zb = zb[::-1]

B = np.full(imax, 1.0, dtype=float)
A = h0*B
Q = q*B

Qup = Q[0]
Adown = A[-1] 

tout = 600
for it in range(int(totalTime/dt)):
    A, Q, err = UnSteadyflowStaggered(A, Q, Adown, Qup, zb, B, dx, dt, g, manning)
    if (dt*it)%tout==0: print((dt*it), err)
        
hc1 = A[::-1]/B[::-1] 

## dx=10.0

In [ ]:
%%time

dx = 10.0
Ls2 = np.arange(0,10000.1,dx)
Zip2 = np.interp(Ls2, df.L, df.Z)

imax = len(Ls2)
dt = 0.025
totalTime = 3.0*3600.0
manning = n

# Initial & Boundary condition
zb = Zip2.copy()
zb = zb[::-1]

B = np.full(imax, 1.0, dtype=float)
A = h0*B
Q = q*B

Qup = Q[0]
Adown = A[-1] 

tout = 600
for it in range(int(totalTime/dt)):
    A, Q, err = UnSteadyflowStaggered(A, Q, Adown, Qup, zb, B, dx, dt, g, manning)
    if (dt*it)%tout==0: print((dt*it), err)
        
hc2 = A[::-1]/B[::-1] 

## dx=100.0

In [ ]:
%%time

dx = 100.0
Ls3 = np.arange(0,10000.1,dx)
Zip3 = np.interp(Ls3, df.L, df.Z)

imax = len(Ls3)
dt = 0.025
totalTime = 3.0*3600.0
manning = n

# Initial & Boundary condition
zb = Zip3.copy()
zb = zb[::-1]

B = np.full(imax, 1.0, dtype=float)
A = h0*B
Q = q*B

Qup = Q[0]
Adown = A[-1] 

tout = 600
for it in range(int(totalTime/dt)):
    A, Q, err = UnSteadyflowStaggered(A, Q, Adown, Qup, zb, B, dx, dt, g, manning)
    if (dt*it)%tout==0: print((dt*it), err)
        
hc3 = A[::-1]/B[::-1] 

# fig

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
fig = hv.Curve((df.L,df.Z), label='Zb dx=0.5m').options(show_grid=True, ylabel='elevation[m]', xlabel='distance[m]' ,color='k') \
* hv.Scatter((Ls2, Zip2), label='Zb dx=10m').options(color='r',size=6) \
* hv.Scatter((Ls3, Zip3), label='Zb dx=100m').options(color='g',size=6) \
* hv.Curve((Ls1, Zip1 + hc1), label='dx=0.5m' ).options(color='b') \
* hv.Curve((Ls2, Zip2 + hc2), label='dx=10m').options(color='r') \
* hv.Curve((Ls3, Zip3 + hc3), label='dx=100m').options(color='g')

In [ ]:
hv.extension('bokeh')
figo = fig.options(width=500, legend_position='bottom_right')
# d = hv.save(figo, 'USSdx.html')
figo 

In [ ]:
hv.extension('bokeh')
figo2 = fig.options(xlim=(3050, 3350), ylim=(7, 12), width=700, height=600, legend_position='right')
# d = hv.save(figo2, 'USSdx2.html')
figo2